In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from inception_unet import bulid_model


## Dice loss

In [2]:
def dic_coeff(y_true,y_hat):
    smooth = 1e-15
    y_true = tf.keras.layers.Flatten()(y_true)
    y_hat = tf.keras.layers.Flatten()(y_hat)
    intersection = tf.reduce_sum(y_hat*y_true)
    return (2*intersection + smooth)/(tf.reduce_sum(y_true) + tf.reduce_sum(y_hat))
    
                                      

In [3]:
def dice_loss(y_true,y_hat):
    return 1 - dic_coeff(y_true,y_hat)

## Read data


In [4]:
dataset_path = r"F:\archive (4)\images"
size = 256
x_data = []
for img in os.listdir(dataset_path):
    img_path = os.path.join(dataset_path, img)
    img_data = cv2.imread(img_path,cv2.IMREAD_COLOR)
    img_data = cv2.resize(img_data,(size,size))
    img_data = img_data/255.0
    img_data = img_data.astype(np.float32)
    x_data.append(img_data)

In [5]:
dataset_path = r"F:\archive (4)\masks"
size = 256
y_data = []
for mask in os.listdir(dataset_path):
    mask_path = os.path.join(dataset_path, mask)
    mask_data = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    mask_data = cv2.resize(mask_data,(size,size))
    mask_data = mask_data/255.0
    mask_data = np.expand_dims(mask_data, axis=-1)
    mask_data = mask_data.astype(np.float32)
    y_data.append(mask_data)    

In [7]:
x_data = np.array([x_data])
y_data = np.array([y_data])

MemoryError: Unable to allocate 2.24 GiB for an array with shape (1, 3064, 256, 256, 3) and data type float32